# AWS Lambda Fashion Classification

- [Notebook](notebooks/python/machine-learning/fashion-classification.md)


Install packages


In [2]:
!uv pip install -q \
    tensorflow==2.16.1 \
    tf2onnx==1.16.1 \
    onnxruntime==1.17.0

Import packages


In [ ]:
import tensorflow as tf
import tf2onnx

print(tf.__version__)

2.16.1


Copy model binary


In [ ]:
!cp ../../machine-learning/xception_v4_37_0.891.h5 .

Load model


In [ ]:
model = tf.keras.models.load_model("./xception_v4_37_0.891.h5", compile=False)

2026-01-04 18:22:40.540469: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-04 18:22:40.550569: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-04 18:22:40.556005: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-04 18:22:41.215791: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.


Convert into ONNX (Open Neural Network Exchange)


In [ ]:
spec = (tf.TensorSpec((None, 299, 299, 3), tf.float32, name="input"),)

model_proto, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=16,
    output_path="clothing-classification.onnx",
)

2026-01-04 18:24:01.123970: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2026-01-04 18:24:01.124307: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2026-01-04 18:24:05.811181: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2026-01-04 18:24:05.811396: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
